In [9]:
import docker 
import tarfile

In [10]:
client = docker.from_env()
device_request = [docker.types.DeviceRequest(count=-2, capabilities=[['gpu']])]
container = client.containers.run("3850639cdf7a", detach=True, stdin_open=True, tty=True, mem_limit="1500m", device_requests=device_request)
print("Container started")

tar = tarfile.open('ClientInput.tar', mode='w')
try:
    tar.add("ModelCode"+".py")
    tar.add("ModelTrainer.py")
finally:
    tar.close()
data = open('ClientInput.tar', 'rb').read()
container.put_archive('/workspace', data)
print("Model sent")

exit_code, output = container.exec_run("python " + "ModelTrainer.py")
print(output.decode())

f = open('ClientOutput.tar', 'wb')
bits, stat = container.get_archive('/workspace/model.pb')
print(stat)
for chunk in bits:
    f.write(chunk)
f.close()

container.stop()
container.remove()

Container started
Model sent
9913344it [00:01, 5599165.74it/s]                             
29696it [00:00, 18416982.34it/s]         
1649664it [00:00, 5369604.89it/s]                             
5120it [00:00, 9882575.46it/s]          
/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs a

In [13]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.utils.data as data
from ModelCode import CNN
import torch.optim as optim


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [14]:
BATCH_SIZE = 1000

train_data = datasets.MNIST(root = 'data', train = True, transform = ToTensor(), download = True)
train_iterator = data.DataLoader(train_data, shuffle = True, batch_size = BATCH_SIZE)

test_data = datasets.MNIST(root = 'data', train = False, transform = ToTensor())
test_iterator = data.DataLoader(test_data, batch_size = BATCH_SIZE)

In [16]:
def train(model, iterator, optimizer, criterion): 
    epoch_loss = 0
    model.train()                
    for (x, y) in iterator:   
        x, y = x.to(device), y.to(device)        
        optimizer.zero_grad()                
        y_pred = model(x) 
        loss = criterion(y_pred, y)         
        loss.backward()              
        optimizer.step()                     
        epoch_loss += loss.item()
    return epoch_loss/len(iterator)         

def evaluate(model, iterator, criterion):      
    epoch_loss = 0
    model.eval()                                 
    with torch.no_grad():                        
        for (x, y) in iterator:  
            x, y = x.to(device), y.to(device)          
            y_pred = model(x)           
            loss = criterion(y_pred, y)      
            epoch_loss += loss.item()
        return epoch_loss/len(iterator) 

def pred(model, iterator):   
    correct = 0
    model.eval()                                 
    with torch.no_grad():                        
        for (x, y) in iterator: 
            x, y = x.to(device), y.to(device)          
            y_pred = model(x)
            y_pred = np.argmax(y_pred.cpu().detach().numpy(), axis=1)
            y = y.cpu().detach().numpy() 
            for i in range(len(y_pred)):
                if y_pred[i] == y[i]:
                    correct += 1
    return correct

In [17]:
from ModelCode import CNN
my_tar = tarfile.open('ClientOutput.tar')
my_tar.extract('model.pb')
my_tar.close()
t1 = torch.load("model.pb")
print(t1)
t1.to(device) 
print("Model loaded on GPU")
print("Test accuracy :",pred(t1, test_iterator)/(BATCH_SIZE*len(test_iterator)))

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)
Model loaded on GPU
Test accuracy : 0.9756
